In [98]:
import sqlite3
from langchain_community.utilities import SQLDatabase
from langchain.llms import Ollama

In [100]:
db_path = "data/demo.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM titanic LIMIT 10;")

sqlite
['titanic']


"[('N', '3', 'Mr. Owen Harris Braund', 'male', '22.0', '1', '0', '7.25'), ('1', '1', 'Mrs. John Bradley (Florence Briggs Thayer) Cumings', 'female', '38.0', '1', '0', '71.2833'), ('1', '3', 'Miss. Laina Heikkinen', 'female', '26.0', '0', '0', '7.925'), ('1', '1', 'Mrs. Jacques Heath (Lily May Peel) Futrelle', 'female', '35.0', '1', '0', '53.1'), ('0', '3', 'Mr. William Henry Allen', 'male', '35.0', '0', '0', '8.05'), ('0', '3', 'Mr. James Moran', 'male', '27.0', '0', '0', '8.4583'), ('0', '1', 'Mr. Timothy J McCarthy', 'male', '54.0', '0', '0', '51.8625'), ('0', '3', 'Master. Gosta Leonard Palsson', 'male', '2.0', '3', '1', '21.075'), ('1', '3', 'Mrs. Oscar W (Elisabeth Vilhelmina Berg) Johnson', 'female', '27.0', '0', '2', '11.1333'), ('1', '2', 'Mrs. Nicholas (Adele Achem) Nasser', 'female', '14.0', '1', '0', '30.0708')]"

In [80]:
# print(db.table_info)


CREATE TABLE titanic (
	"Survived" TEXT, 
	"Pclass" TEXT, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" TEXT, 
	"Siblings_Spouses_Aboard" TEXT, 
	"Parents_Children_Aboard" TEXT, 
	"Fare" TEXT
)

/*
3 rows from titanic table:
Survived	Pclass	Name	Sex	Age	Siblings_Spouses_Aboard	Parents_Children_Aboard	Fare
N	3	Mr. Owen Harris Braund	male	22.0	1	0	7.25
1	1	Mrs. John Bradley (Florence Briggs Thayer) Cumings	female	38.0	1	0	71.2833
1	3	Miss. Laina Heikkinen	female	26.0	0	0	7.925
*/


In [118]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents import AgentType

print(agent_executor.get_prompts)

<bound method Runnable.get_prompts of AgentExecutor(name='SQL Agent Executor', verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': "sql_db_query - Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.\nsql_db_schema - Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3\nsql_db_list_tables - Input is an empty string, output 

In [124]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate

system_prefix = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer.
"""

system_suffix = """Frame the answer into a sentence and return it in an easy way to understand"""

agent_executor = create_sql_agent(llm, db=db, prefix=system_prefix, suffix=system_suffix, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
# agent_executor.invoke({"input": "Describe the titanic table"})

In [126]:
result=agent_executor.invoke("How many passengers were present in the Titanic ?")



> Entering new SQL Agent Executor chain...
 Thought: First, I need to ensure that there is a table related to the Titanic's passenger information. If not, I will check if such tables exist in the database using sql_db_list_tables tool. Then, after confirming the existence of relevant tables and their schema with sql_db_schema, I can construct an SQL query to count passengers on the Titanic.

Action: sql_db_list_tables
Action Input: titanic Thought: Now that I have confirmed the existence of a table related to the Titanic's passenger information, I will use sql_db_schema tool to get the schema and sample rows for this table. After understanding the structure of the table, I can construct an SQL query using sql_db_query_checker to ensure its correctness before executing it with sql_db_query.

Action: sql_db_schema
Action Input: titanic
CREATE TABLE titanic (
	"Survived" TEXT, 
	"Pclass" TEXT, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" TEXT, 
	"Siblings_Spouses_Aboard" TEXT, 
	"Parents_Childr

In [92]:
result=agent_executor.invoke("Who are you ?")



> Entering new SQL Agent Executor chain...
 Question: Who are you?
Thought: This is a non-technical question. As an AI developed by Microsoft, I am a digital assistant designed to help with information and tasks using my capabilities in processing data and providing helpful responses based on the knowledge encoded within me up until my last update.

Final Answer: I am a digital assistant powered by artificial intelligence, here to provide you with assistance and information.

(Note: Since this is not an SQL-related query, there's no need for any database action tools in this context.)

> Finished chain.


In [128]:
result=agent_executor.invoke("Describe the titanic table ?")



> Entering new SQL Agent Executor chain...
 Action: sql_db_list_tables
Action Input: titanic Thought: Now that I have confirmed the existence of the "titanic" table in the database, I should use the sql_db_schema tool to get its schema and sample rows.
Action: sql_db_schema
Action Input: titanic
CREATE TABLE titanic (
	"Survived" TEXT, 
	"Pclass" TEXT, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" TEXT, 
	"Siblings_Spouses_Aboard" TEXT, 
	"Parents_Children_Aboard" TEXT, 
	"Fare" TEXT
)

/*
3 rows from titanic table:
Survived	Pclass	Name	Sex	Age	Siblings_Spouses_Aboard	Parents_Children_Aboard	Fare
N	3	Mr. Owen Harris Braund	male	22.0	1	0	7.25
1	1	Mrs. John Bradley (Florence Briggs Thayer) Cumings	female	38.0	1	0	71.2833
1	3	Miss. Laina Heikkinen	female	26.0	0	0	7.925
*/ Final Answer: The Titanic table is a database table that contains information about the passengers of the Titanic ship. It has the following columns and their respective data types:

1. Survived - Text (indicates if the passeng

In [130]:
llm

Ollama(model='phi3')

In [138]:
llm = Ollama(model="sample")
print(llm.invoke("Who are you ?"))

 I'm TechHelperBot, created as a Research project at the Technical University of Ilmenau under Jijo Valiyaveettil for assisting with queries regarding our unique 5G infrastructure inside the institution! How can I help today? Are there specific details about our configurations you need information on within the scope mentioned above (base station locations, bandwidth, transmission power, antenna types and configuration, network coverage area, signal strength)? Any upgrades or issues to discuss in relation with 5G connectivity at the University of Ilmenau. Let'this interaction about providing insights into our advanced communication system!


In [ ]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# Create the SQL query chain
chain = create_sql_query_chain(llm, db)

# Define the prompt template
answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

# Define the final chain with the query and result processing
execute_query = QuerySQLDataBaseTool(db=db)
answer = answer_prompt | llm | StrOutputParser()

final_chain = (
    RunnablePassthrough.assign(query=lambda x: x['sql_query']).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

def ask_question(question):
    # Invoke the chain to get the SQL query
    response = chain.invoke({"question": question})

    # Extract the SQL query from the response
    sql_query = response.strip().split(': ')[2]

    # Get the final answer
    result = final_chain.invoke({"question": question, "sql_query": sql_query})
    
    return result